In [3]:
from __future__ import print_function
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler

In [4]:
if __name__ == "__main__":
    #Create a sparkSession (Note, the config section is only for windows!)
    spark = SparkSession.builder.appName ("DecisionTree Regressor").getOrCreate()
    #Load up our data and convert it to the format Lib expects. Inputlines spark.
    #data =spark.read.csv ("fille:///sparkCourse/realestate.csv")
    lines = spark.read.option ("header","true").option("inferSchema","true").csv("file:///sparkCourse/realestate.csv")
    inputLine = VectorAssembler().setInputCols(["DistanceToMRT","NumberConvenienceStores"]).setOutputCol("features")
    
    df = inputLine.transform(lines).select("PriceOfUnitArea","features")
    
    trainTest = df.randomSplit([0.5,0.5])
    trainingDF = trainTest[0]
    testDF = trainTest[1]
    
    dtr = DecisionTreeRegressor().setFeaturesCol("features").setLabelCol("PriceOfUnitArea")
    
    model = dtr.fit(trainingDF)
    
    
    fullPredictions = model.transform(testDF).cache()
    
    predictions = fullPredictions.select("prediction").rdd.map(lambda x : x[0])
    labels = fullPredictions.select("PriceOfUnitArea").rdd.map(lambda x : x[0])
    
    predictionAndLabel = predictions.zip(labels).collect()
    
    for prediction in predictionAndLabel:
        print(prediction)
        
    spark.stop()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 1 times, most recent failure: Lost task 0.0 in stage 16.0 (TID 16) (DESKTOP-OD3AURM executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.ZippedPartitionsRDD2.compute(ZippedPartitionsRDD.scala:89)


In [35]:
fullPredictions.show()

+---------------+--------------+------------------+
|PriceOfUnitArea|      features|        prediction|
+---------------+--------------+------------------+
|           11.6|[4066.587,0.0]|20.183333333333334|
|           12.2|[6396.283,1.0]|14.299999999999997|
|           12.8|[1156.777,0.0]|             19.98|
|           12.8|[4082.015,0.0]|17.119999999999997|
|           13.0|[4197.349,0.0]|17.119999999999997|
|           13.2|[4510.359,1.0]|14.299999999999997|
|           13.4|[4605.749,0.0]|17.825000000000003|
|           13.8|[1159.454,0.0]|             19.98|
|           15.0|[6306.153,1.0]|14.299999999999997|
|           15.6|[4082.015,0.0]|17.119999999999997|
|           16.7|[4082.015,0.0]|17.119999999999997|
|           18.3|[4066.587,0.0]|20.183333333333334|
|           18.6|[4197.349,0.0]|17.119999999999997|
|           18.8|[5512.038,1.0]|14.299999999999997|
|           19.0|[1156.412,0.0]|             19.98|
|           19.2|[4197.349,0.0]|17.119999999999997|
|           